#### Union of two departments_a and departments_b write result output to Silver lakehouse

In [9]:
from pyspark.sql.functions import col,lit,concat
df_hosa=spark.read.table('departments_a')
df_hosb=spark.read.table('departments_b')
df_union=(df_hosa.withColumn('data_source',lit('hosa'))).unionByName(df_hosb.withColumn('data_source',lit('hosb')))
#display(df_union.head(3))
df_union=df_union.withColumn("Src_deptid",col('DeptID'))\
                    .withColumn('Dept_id',concat(col('DeptID'),lit('_'),col('data_source'))).drop('DeptID')
df_union.write.format('delta').mode('overwrite').saveAsTable('Silver_LH.departments')


StatementMeta(, 314bf14b-aa99-4912-85f5-1ef54d994324, 11, Finished, Available, Finished)

##### Querying Tables

In [2]:
%%sql
select * from BronzeLH.departments_a limit 5;
select * from BronzeLH.departments_b limit 5;



StatementMeta(, 654ad980-08b7-4ce5-bdcb-d4e16facfa5f, 5, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 2 fields>

<Spark SQL result set with 5 rows and 2 fields>

##### Querying Tables

In [11]:
%%sql
SELECT * from Silver_LH.departments;

StatementMeta(, 314bf14b-aa99-4912-85f5-1ef54d994324, 13, Finished, Available, Finished)

<Spark SQL result set with 40 rows and 4 fields>

#### Union of two providers_a and providers_b write result output to Silver lakehouse

In [19]:
from pyspark.sql.functions import col,lit,concat
df_providers_a=spark.read.table('BronzeLH.providers_a')
df_providers_b=spark.read.table('BronzeLH.providers_b')
df_union=(df_providers_a.withColumn('data_source',lit('hosa'))).unionByName(df_providers_b.withColumn('data_source',lit('hosb')))
df_union.distinct().write.format('delta').mode('overwrite').saveAsTable('Silver_LH.providers')


StatementMeta(, 314bf14b-aa99-4912-85f5-1ef54d994324, 22, Finished, Available, Finished)

##### Querying Tables

In [12]:
%%sql
select * from BronzeLH.providers_a limit 3;
select * from BronzeLH.providers_b limit 3;

StatementMeta(, 314bf14b-aa99-4912-85f5-1ef54d994324, 15, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 6 fields>

<Spark SQL result set with 3 rows and 6 fields>

##### DDL patients delta table in silver lakehouse

In [42]:
%%sql
CREATE TABLE IF NOT EXISTS Silver_LH.patients (
    Patient_Key STRING,
    SRC_PatientID STRING,
    FirstName STRING,
    LastName STRING,
    MiddleName STRING,
    SSN STRING,
    PhoneNumber STRING,
    Gender STRING,
    DOB DATE,
    Address STRING,
    SRC_ModifiedDate TIMESTAMP,
    datasource STRING,
    is_quarantined BOOLEAN,
    inserted_date TIMESTAMP,
    modified_date TIMESTAMP,
    is_current BOOLEAN
)
USING DELTA;

StatementMeta(, 314bf14b-aa99-4912-85f5-1ef54d994324, 45, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### Querying Tables

In [21]:
%%sql
select * from Silver_LH.patients

StatementMeta(, b5444943-9d24-437b-979b-51b3bde2c32e, 23, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 16 fields>

##### Querying Tables

In [13]:
%%sql
select * from patient_data limit 10 

StatementMeta(, b5444943-9d24-437b-979b-51b3bde2c32e, 15, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 13 fields>

#### Union of two patients_a and patients_b write result output to Silver lakehouse

In [18]:
from pyspark.sql.functions import col,lit,concat,when
df_patients_a=spark.read.table('BronzeLH.patients_a')
df_patients_b=spark.read.table('BronzeLH.patients_b')
#df_patients_a.printSchema()
#df_patients_b.printSchema()
df_patients_b=df_patients_b.withColumnRenamed('ID','PatientID').withColumnRenamed('F_Name','FirstName').withColumnRenamed('L_Name','LastName')\
.withColumnRenamed('M_Name','MiddleName').withColumnRenamed('Updated_Date','ModifiedDate')
df_union=(df_patients_a.withColumn('data_source',lit('hosa'))).unionByName(df_patients_b.withColumn('data_source',lit('hosb')))
df_cdm_patients=df_union.withColumn('SRC_PatientID',concat(col('PatientID'),lit('_'),col('data_source')))\
.withColumn('is_valid' ,when(col('PatientID').isNull() or col('FirstName').isNull() or col('DOB').isNull(),'False').otherwise('True'))
#df_cdm_patients.printSchema()
#df_cdm_patients.createOrReplaceTempView("source")




StatementMeta(, b5444943-9d24-437b-979b-51b3bde2c32e, 20, Finished, Available, Finished)

##### Querying Tables

In [11]:
spark.sql("SHOW TABLES").show()


StatementMeta(, b5444943-9d24-437b-979b-51b3bde2c32e, 13, Finished, Available, Finished)

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|silver_lh|      ext_tb|      false|
|silver_lh|  fact_table|      false|
|silver_lh| departments|      false|
|silver_lh|   providers|      false|
|silver_lh|    patients|      false|
|         |patient_data|      false|
+---------+------------+-----------+



##### creating Delta table must be registered 

In [2]:
from delta.tables import DeltaTable
Silver_LH_source = DeltaTable.forName(spark, "patients")

StatementMeta(, b5444943-9d24-437b-979b-51b3bde2c32e, 4, Finished, Available, Finished)

#### Merge tables from patients source to target table

In [20]:
Silver_LH_source.alias('Target').merge(
    source=df_cdm_patients.alias('source'),
    condition=(
        'Target.Patient_Key = source.PatientID' and
        'Target.is_valid = True' and (
            'Target.SRC_PatientID <> source.SRC_PatientID' or
            'Target.FirstName <> source.FirstName' or
            'Target.LastName <> source.LastName' or
            'Target.MiddleName <> source.MiddleName' or
            'Target.SSN <> source.SSN' or
            'Target.PhoneNumber <> source.PhoneNumber' or
            'Target.Gender <> source.Gender' or
            'Target.DOB <> source.DOB' or
            'Target.Address <> source.Address' or
            'Target.SRC_ModifiedDate <> source.ModifiedDate' or
            'Target.datasource <> source.data_source' or
            'Target.is_quarantined <> source.is_quarantined'
        )
    )
).whenMatchedUpdate(
    set={
        'Target.is_current': 'False',
        'Target.modified_date': 'current_timestamp()'
    }
).whenNotMatchedInsert(
    values={
        'Patient_Key': 'source.PatientID',
        'SRC_PatientID': 'source.SRC_PatientID',
        'FirstName': 'source.FirstName',
        'LastName': 'source.LastName',
        'MiddleName': 'source.MiddleName',
        'SSN': 'source.SSN',
        'PhoneNumber': 'source.PhoneNumber',
        'Gender': 'source.Gender',
        'DOB': 'source.DOB',
        'Address': 'source.Address',
        'SRC_ModifiedDate': 'source.ModifiedDate',
        'datasource': 'source.data_source',
        'is_quarantined': 'source.is_valid',
        'inserted_date': 'current_timestamp()',
        'modified_date': 'current_timestamp()',
        'is_current': 'True'
    }
).execute()


StatementMeta(, b5444943-9d24-437b-979b-51b3bde2c32e, 22, Finished, Available, Finished)

#### Union of two transactions_a and transactions_b write result output to Silver lakehouse

In [2]:
from pyspark.sql.functions import col,lit,concat,when
df_transactions_a=spark.read.table('BronzeLH.transactions_a')
df_transactions_b=spark.read.table('BronzeLH.transactions_b')
df_union_transactions=(df_transactions_a.withColumn('data_source',lit('hosa')).withColumn('SRC_TransactionID',concat('TransactionID',lit('_'),lit('hosa')))).unionByName(df_transactions_b.withColumn('data_source',lit('hosb')).withColumn('SRC_TransactionID',concat('TransactionID',lit('_'),lit('hosb'))))
df_union_transaction=df_union_transactions \
.withColumn('is_quarantined',\
when(col('TransactionID').isNull() or col('EncounterID').isNull()or col('VisitDate').isNull(),'True').otherwise('False'))
#display(df_union_transaction.head(4))


StatementMeta(, 806f840e-0b4a-44b5-9124-488a3ef83999, 4, Finished, Available, Finished)

#### DDL transactions table

In [2]:
%%sql
CREATE TABLE IF NOT EXISTS Silver_LH.transactions (
  TransactionID string,
  SRC_TransactionID string,
  EncounterID string,
  PatientID string,
  ProviderID string,
  DeptID string,
  VisitDate date,
  ServiceDate date,
  PaidDate date,
  VisitType string,
  Amount double,
  AmountType string,
  PaidAmount double,
  ClaimID string,
  PayorID string,
  ProcedureCode integer,
  ICDCode string,
  LineOfBusiness string,
  MedicaidID string,
  MedicareID string,
  SRC_InsertDate date,
  SRC_ModifiedDate date,
  datasource string,
  is_quarantined boolean,
  audit_insertdate timestamp,
  audit_modifieddate timestamp,
  is_current boolean
)
USING DELTA;

StatementMeta(, fb306b53-13f9-495f-b38c-67a82b93f43a, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### querying tables

In [35]:
%%sql
select * from Silver_LH.Silver_LH_transactions_source;

StatementMeta(, fb306b53-13f9-495f-b38c-67a82b93f43a, 40, Finished, Available, Finished)

Error: [TABLE_OR_VIEW_NOT_FOUND] The table or view `Silver_LH`.`Silver_LH_transactions_source` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [Silver_LH, Silver_LH_transactions_source], [], false


##### creating Delta table must be registered 

In [3]:
from delta.tables import DeltaTable
Silver_LH_transactions_source = DeltaTable.forName(spark, "transactions")

StatementMeta(, 806f840e-0b4a-44b5-9124-488a3ef83999, 5, Finished, Available, Finished)

#### Quering tables

In [1]:
%%sql
select * from Silver_LH.transactions limit 5;


StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 2, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 27 fields>

#### Merge tables from transactions source to target table

In [6]:
from pyspark.sql.functions import current_timestamp
Silver_LH_transactions_source.alias('Target').merge(source=df_union_transaction.alias('source'),
condition=('Target.TransactionID=source.TransactionID' and 'Target.is_current=True' and 
('Target.SRC_TransactionID <> source.SRC_TransactionID' or
'Target.EncounterID <> source.EncounterID' or
'Target.PatientID <> source.PatientID' or
'Target.ProviderID <> source.ProviderID' or
'Target.DeptID <> source.DeptID' or
'Target.VisitDate <> source.VisitDate' or
'Target.ServiceDate <> source.ServiceDate' or
'Target.PaidDate <> source.PaidDate' or
'Target.VisitType <> source.VisitType' or
'Target.Amount <> source.Amount' or
'Target.AmountType <> source.AmountType' or
'Target.PaidAmount <>source.PaidAmount' or
'Target.ClaimID <> source.ClaimID' or
'Target.PayorID <> source.PayorID' or
'Target.ProcedureCode <> source.ProcedureCode' or
'Target.ICDCode <> source.ICDCode' or
'Target.LineOfBusiness <> source.LineOfBusiness' or
'Target.MedicalID <> source.MedicalID' or
'Target.MedicareID <> source.MedicareID' or
'Target.InsertDate <> source.InsertDate' or
'Target.ModifiedDate <> source.ModifiedDate'))).whenMatchedUpdate(
set={
    'Target.audit_modifieddate' : 'current_timestamp()' ,
    'Target.is_current':'False'
}
).whenNotMatchedInsert(\
values={
'TransactionID' : 'source.TransactionID',
'SRC_TransactionID' : 'source.SRC_TransactionID',
'EncounterID' : 'source.EncounterID',
'PatientID' : 'source.PatientID',
'ProviderID' : 'source.ProviderID',
'DeptID' : 'source.DeptID',
'VisitDate' : 'source.VisitDate',
'ServiceDate' : 'source.ServiceDate',
'PaidDate' : 'source.PaidDate',
'VisitType' : 'source.VisitType',
'Amount' : 'source.Amount',
'AmountType' : 'source.AmountType',
'PaidAmount' : 'source.PaidAmount',
'ClaimID' : 'source.ClaimID',
'PayorID' : 'source.PayorID',
'ProcedureCode' : 'source.ProcedureCode',
'ICDCode' : 'source.ICDCode',
'LineOfBusiness' : 'source.LineOfBusiness',
'MedicaidID' : 'source.MedicaidID',
'MedicareID' : 'source.MedicareID',
'SRC_InsertDate' : 'source.InsertDate',
'SRC_ModifiedDate' : 'source.ModifiedDate',
'datasource' : 'source.data_source',
'is_quarantined' : 'source.is_quarantined',
'audit_insertdate' : 'current_timestamp()',
'audit_modifieddate' : 'current_timestamp()',
'is_current' : 'True'   
}
).execute()

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 8, Finished, Available, Finished)

NameError: name 'Silver_LH_transactions_source' is not defined

#### Quering tables

In [1]:
%%sql
select * from BronzeLH.transactions_a limit 4;
select * from BronzeLH.transactions_b limit 4;

StatementMeta(, fb306b53-13f9-495f-b38c-67a82b93f43a, 3, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 21 fields>

<Spark SQL result set with 4 rows and 21 fields>

#### Quering tables

In [11]:
%%sql
select * from encounters limit 5;

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 13, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 15 fields>

#### Merge tables from encounters source to target table

In [10]:
from pyspark.sql.functions import current_timestamp
Silver_LH_encounters_source.alias('Target').merge(source=df_union_encounter.alias('source'),
condition='Target.EncounterID=source.EncounterID' and 'Target.is_current=True' and
('Target.SRC_EncounterID <> source.SRC_EncounterID' or
'Target.PatientID <> source.PatientID' or
'Target.EncounterDate <> source.EncounterDate' or
'Target.EncounterType <> source.EncounterType' or
'Target.ProviderID <> source.ProviderID' or
'Target.DepartmentID <> source.DepartmentID' or
'Target.ProcedureCode <> source.ProcedureCode' or
'Target.SRC_InsertedDate <> source.SRC_InsertedDate' or
'Target.SRC_ModifiedDate <> source.SRC_ModifiedDate'or
'Target.datasource <> source.data_source' or
'Target.is_quarantined <> source.is_quarantined')).whenMatchedUpdate(set=
{
'Target.audit_modifieddate':'current_timestamp()',
'Target.is_current':'True'
}).whenNotMatchedInsert(
    values={
'EncounterID' : 'source.EncounterID',
'SRC_EncounterID' : 'source.SRC_EncounterID',
'PatientID' : 'source.PatientID',
'EncounterDate' : 'source.EncounterDate',
'EncounterType' : 'source.EncounterType',
'ProviderID' : 'source.ProviderID',
'DepartmentID' : 'source.DepartmentID',
'ProcedureCode' : 'source.ProcedureCode',
'SRC_InsertedDate' : 'source.InsertedDate',
'SRC_ModifiedDate' : 'source.ModifiedDate',
'datasource' : 'source.data_source',
'is_quarantined' : 'source.is_quarantined',
'audit_insertdate' : 'current_timestamp()',
'audit_modifieddate' : 'current_timestamp()',
'is_current' : 'true'

    }
).execute()

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 12, Finished, Available, Finished)

##### creating Delta table must be registered 

In [4]:
from delta.tables import DeltaTable
Silver_LH_encounters_source = DeltaTable.forName(spark, "encounters")

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 6, Finished, Available, Finished)

#### Quering tables

In [3]:
from pyspark.sql.functions import col,lit,concat,when
df_encounters_a=spark.read.table('BronzeLH.encounters_a')
df_encounters_b=spark.read.table('BronzeLH.encounters_b')
df_union_encounters=(df_encounters_a.withColumn('data_source',lit('hosa')).withColumn('SRC_EncounterID',concat('EncounterID',lit('_'),lit('hosa')))).unionByName(df_encounters_b.withColumn('data_source',lit('hosb')).withColumn('SRC_EncounterID',concat('EncounterID',lit('_'),lit('hosb'))))
df_union_encounter=df_union_encounters\
.withColumn('is_quarantined',\
when(col('EncounterID').isNull()or col('PatientID').isNull(),'True').otherwise('False'))
#display(df_union_encounter.head(4))

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, faa064a2-f155-4bd7-a3d6-4e5ac429164f)

#### DDL table Encounters

In [2]:
%%sql
CREATE TABLE IF NOT EXISTS Silver_LH.Encounters (
    EncounterID string,
    SRC_EncounterID string,
    PatientID string,
    EncounterDate date,
    EncounterType string,
    ProviderID string,
    DepartmentID string,
    ProcedureCode integer,
    SRC_InsertedDate date,
    SRC_ModifiedDate date,
    datasource string,
    is_quarantined boolean,
    audit_insertdate timestamp,
    audit_modifieddate timestamp,
    is_current boolean
)
USING DELTA;

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

#### Quering tables

In [32]:
%%sql
select * from Silver_LH.cptcodes limit 5;

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 34, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 8 fields>

##### creating Delta table must be registered 

In [30]:
from delta.tables import DeltaTable
Silver_LH_cpt_codes_source = DeltaTable.forName(spark, "cptcodes")

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 32, Finished, Available, Finished)

#### Merge tables from cptcodes source to target table

In [31]:
from pyspark.sql.functions import current_timestamp
Silver_LH_cpt_codes_source.alias('Target').merge(source=df_cpt_codes.alias('source'),
condition='Target.cpt_codes=source.cpt_codes' and 'Target.is_current=True').whenMatchedUpdate(set=
{
    'Target.audit_modifieddate':'current_timestamp()','Target.is_current':'true'
}).whenNotMatchedInsert( values=
{
'cpt_codes' : 'source.cpt_codes',
'procedure_code_category' : 'source.procedure_code_category',
'procedure_code_descriptions' : 'source.procedure_code_descriptions',
'code_status' : 'source.code_status',
'is_quarantined' : 'source.is_quarantined',
'audit_insertdate' : 'current_timestamp()',
'audit_modifieddate' : 'current_timestamp()',
'is_current' : 'true'
}).execute()

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 33, Finished, Available, Finished)

In [29]:
from pyspark.sql.functions import when,col,lit
df_cpt_codes=spark.read.table('BronzeLH.cpt_codes')
df_cpt_codes=df_cpt_codes.withColumn('is_quarantined',when(col('cpt_codes').isNull() | col('procedure_code_descriptions').isNull(),'True').otherwise('False'))
#display(df_cpt_codes.head(4))

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 689cd9d8-95cb-441a-ab43-c584d57683df)

##### DDL cptcodes delta table in silver lakehouse

In [14]:
%%sql
CREATE TABLE IF NOT EXISTS Silver_LH.cptcodes (
    cpt_codes string,
    procedure_code_category string,
    procedure_code_descriptions string,
    code_status string,
    is_quarantined boolean,
    audit_insertdate timestamp,
    audit_modifieddate timestamp,
    is_current boolean
)
USING DELTA

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 16, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

#### Quering tables

In [44]:
%%sql
select * from Silver_LH.claims limit 5;

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 46, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 24 fields>

#### Merge tables from claims source to target table

In [43]:
from pyspark.sql.functions import current_timestamp
Silver_LH_claims_source.alias('Target').merge(source=df_claims.alias('source'),
condition='Target.ClaimID =source.ClaimID' and 'Target.is_current=true').whenMatchedUpdate(set={
'Target.audit_modifieddate':'current_timestamp()','Target.is_current':'true'
}).whenNotMatchedInsert(
    values={
'ClaimID':'source.ClaimID',
'SRC_ClaimID':'source.SRC_ClaimID',
'TransactionID':'source.TransactionID',
'PatientID':'source.PatientID',
'EncounterID':'source.EncounterID',
'ProviderID':'source.ProviderID',
'DeptID':'source.DeptID',
'ServiceDate':'source.ServiceDate',
'ClaimDate':'source.ClaimDate',
'PayorID':'source.PayorID',
'ClaimAmount':'source.ClaimAmount',
'PaidAmount':'source.PaidAmount',
'ClaimStatus':'source.ClaimStatus',
'PayorType':'source.PayorType',
'Deductible':'source.Deductible',
'Coinsurance':'source.Coinsurance',
'Copay':'source.Copay',
'SRC_InsertDate':'source.InsertDate',
'SRC_ModifiedDate':'source.ModifiedDate',
'datasource':'source.data_source',
'is_quarantined':'source.is_quarantined',
'audit_insertdate':'current_timestamp()',
'audit_modifieddate':'current_timestamp()',
'is_current':'true'
    }
).execute()

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 45, Finished, Available, Finished)

##### creating Delta table must be registered 

In [41]:
from delta.tables import DeltaTable
Silver_LH_claims_source = DeltaTable.forName(spark, "claims")

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 43, Finished, Available, Finished)

In [40]:
from pyspark.sql.functions import when,col,lit
df_claims=spark.read.table('BronzeLH.claim_data')
df_claims=df_claims.withColumn('is_quarantined',when(col('ClaimID').isNull() | col('TransactionID').isNull()| col('PatientID').isNull(),'True').otherwise('False')).withColumn('SRC_ClaimID',concat(col('ClaimID'),lit('_'),col('data_source')))
display(df_claims.head(4))

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 42, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e9d43787-5f7a-4171-ac30-fd6147851bec)

##### DDL claims delta table in silver lakehouse

In [35]:

%%sql
CREATE TABLE IF NOT EXISTS Silver_LH.claims (
    ClaimID string,
    SRC_ClaimID string,
    TransactionID string,
    PatientID string,
    EncounterID string,
    ProviderID string,
    DeptID string,
    ServiceDate date,
    ClaimDate date,
    PayorID string,
    ClaimAmount string,
    PaidAmount string,
    ClaimStatus string,
    PayorType string,
    Deductible string,
    Coinsurance string,
    Copay string,
    SRC_InsertDate date,
    SRC_ModifiedDate date,
    datasource string,
    is_quarantined boolean,
    audit_insertdate timestamp,
    audit_modifieddate timestamp,
    is_current boolean
)
USING DELTA;

StatementMeta(, fe71d699-9658-4ed6-a9c8-4821b5c5e281, 37, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### DDL departments delta table in silver lakehouse

In [1]:
%%sql
CREATE TABLE IF NOT EXISTS Silver_LH.departments (
Dept_Id string,
SRC_Dept_Id string,
Name string,
datasource string,
is_quarantined boolean
 )
USING DELTA;

StatementMeta(, 60bfd18f-1e33-44b4-9994-b871803d91f2, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

#### Quering tables

In [3]:
%%sql
select * from Silver_LH.providers limit 4;

StatementMeta(, d8192cc5-4fb0-441c-9051-d4242e3fd6cb, 5, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 7 fields>

##### DDL providers delta table in silver lakehouse

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS Silver_LH.providers (
ProviderID string,
FirstName string,
LastName string,
Specialization string,
DeptID string,
NPI long,
data_source string
)
USING DELTA;